# Pure State Tomography

Junling Long, Adrian Tan, Sven Jandura

 Suppose we have some quantum circuit, which generates a state |psi>. We are interested in the coefficients of psi.
 Unfortunatly, due to noise effects, on a real quantum computer we can only generate rho = (1-epsilon)|psi><psi| + epsilon rho_error
 with epsilon << 1. How do we find a good estimate of |psi>

 Using usual quantum state tomography, we can could find rho and would estimate |psi> to be the eigenvector of rho which belongs to
 the larges eigenvalue. Using pure state tomography, we can find an estimate of |psi> directly from the measurements. This requires 
 fewer operators to be measured then for a full state tomography for the same accuraccy.


In [5]:
import numpy as np

import qiskit
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, Aer
from qiskit.quantum_info import state_fidelity
from qiskit.providers.aer import noise
import qiskit.ignis.verification.tomography as tomo

from qiskit import Aer, IBMQ, execute
from qiskit.providers.aer import noise

from qiskit.ignis.verification.tomography.fitters.pure_state_mle_fit import pure_state_mle_fit, pure_state_mle_fit_density_matrix

from scipy.linalg import sqrtm


/usr/lib/python3.7/site-packages/marshmallow/schema.py:364: ChangedInMarshmallow3Warning: strict=False is not recommended. In marshmallow 3.0, schemas will always be strict. See https://marshmallow.readthedocs.io/en/latest/upgrading.html#schemas-are-always-strict
  ChangedInMarshmallow3Warning


## Simple Density Matrix

Consider the Bell states |psi1> = 1/sqrt(2)(|00>+|11>) and |psi2> = 1/sqrt(2)(|00>-|11>) and rho = (1-epsilon)|psi1><psi1| + epsilon|psi2><psi2|



In [6]:
#Psi1
# Setup circuit to generate |psi1>
qr1 = QuantumRegister(2)
bell1 = QuantumCircuit(qr1)
bell1.h(qr1[0])
bell1.cx(qr1[0], qr1[1])

# get state |psi1> using statevector_simulator
job = qiskit.execute(bell1, Aer.get_backend('statevector_simulator'))
psi1 = job.result().get_statevector(bell1)
print("|psi1> = {}".format(psi1))

# get expectation values of operators ... and convert the to the right format
qst_bell1 = tomo.state_tomography_circuits(bell1, qr1)
job = qiskit.execute(qst_bell1, Aer.get_backend('qasm_simulator'), shots=5000)
tomo_counts_psi1 = tomo.tomography_data(job.result(), qst_bell1, efficient = True)
probs_psi1, basis_matrix, _ = tomo.fitter_data(tomo_counts_psi1)

#Psi 2
# Setup circuit to generate |psi2>
qr2 = QuantumRegister(2)
bell2 = QuantumCircuit(qr2)
bell2.y(qr2[0])
bell2.h(qr2[0])
bell2.cx(qr2[0], qr2[1])

# get state |psi2> using statevector_simulator
job = qiskit.execute(bell2, Aer.get_backend('statevector_simulator'))
psi2 = job.result().get_statevector(bell2)
print("|psi2> = {}".format(psi2))


# get expectation values of operators ... and convert the to the right format
qst_bell2 = tomo.state_tomography_circuits(bell2, qr2)
job = qiskit.execute(qst_bell2, Aer.get_backend('qasm_simulator'), shots=5000)
tomo_counts_psi2 = tomo.tomography_data(job.result(), qst_bell2, efficient = True)
probs_psi2, _ , _ = tomo.fitter_data(tomo_counts_psi2) # the basis_matrix is the same for both states


|psi1> = [0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]
|psi2> = [0.+0.70710678j 0.+0.j         0.+0.j         0.-0.70710678j]


Generate probabilities of rho

In [7]:
epsilon = 0.3
probs_rho = [(1-epsilon)*probs_psi1[i]+epsilon*probs_psi2[i] for i in range(len(probs_psi1))]

function for calculating inner product between states


In [8]:
def inner_product(psi, phi):
    return np.abs(sum([psi[i].conj()*phi[i] for i in range(len(psi))]))**2

Full state tomography (maximum likelyhood approach)


In [9]:
rho_full_mle = tomo.state_mle_fit(probs_rho, basis_matrix)
eigenvalues, eigenvectors = np.linalg.eig(rho_full_mle)
print("Eigenvalues of rho: {}".format(eigenvalues))
print("Guess for psi: {}".format(eigenvectors[0]))
print("Fidelity: {}".format(inner_product(psi1, eigenvectors[0])))

Eigenvalues of rho: [ 5.66699285e-01+2.88739156e-18j  3.66648747e-01+1.26582635e-18j
 -4.16493426e-18+3.99787554e-20j  6.66519672e-02+1.66149507e-18j]
Guess for psi: [ 0.70928483+0.j         -0.7047712 -0.01153259j  0.00198461+0.00727256j
 -0.00330089+0.00344293j]
Fidelity: 0.2492125898935742


Pure state tomography (maximum likelyhood approach)


In [10]:
psi_guess,_ = pure_state_mle_fit(probs_rho, basis_matrix)
print("Guess for psi: {}".format(psi_guess))
print("Fidelity: {}".format(inner_product(psi1, psi_guess)))


Guess for psi: [(0.707325922748363-0.0002991421570430552j), (-0.0007223700961176263+0.004682163712799221j), (-0.002903498638227325+0.005640840724021371j), (0.7067418140029688-0.011969293961411297j)]
Fidelity: 0.9998690393229133


## Running on a noise simulator quantum computer

Setup random circuit

In [11]:
qr = QuantumRegister(2)
circuit = QuantumCircuit(qr)
params = np.random.rand(12)
circuit.u3(params[0], params[1], params[2], qr[0])
circuit.u3(params[3], params[4], params[5], qr[1])
circuit.cx(qr[0], qr[1])
circuit.u3(params[6], params[7], params[8], qr[0])
circuit.u3(params[9], params[10], params[11], qr[1])


job = qiskit.execute(circuit, Aer.get_backend('statevector_simulator'))
psi = job.result().get_statevector(circuit)
print("psi: {}".format(psi))

psi: [ 0.96330278-0.00345187j  0.1343249 +0.04397251j  0.19890271+0.02646879j
 -0.07215715+0.08117591j]


Setup Noise model

In [12]:
#IBMQ.load_accounts()
IBMQ.enable_account('a6140115a9d2692b8a711d0f31fdc92b7ae793865719445a78fc210220de52765da18d0eab774d245fbc9e5f40c94e99d9c536f9f1749acd7b902b3bd9931dd5')
device = IBMQ.get_backend('ibmqx4')
properties = device.properties()
coupling_map = device.configuration().coupling_map

gate_times = [
    ('u1', None, 0), ('u2', None, 100), ('u3', None, 200),
    ('cx', [1, 0], 678)]

# Construct the noise model from backend properties
# and custom gate times
noise_model = noise.device.basic_device_noise_model(properties, gate_times=gate_times)


# Get the basis gates for the noise model
basis_gates = noise_model.basis_gates

# Select the QasmSimulator from the Aer provider
simulator = Aer.get_backend('qasm_simulator')


Execute

In [13]:
qst= tomo.state_tomography_circuits(circuit, qr)
job = qiskit.execute(qst, Aer.get_backend('qasm_simulator'),noise_model=noise_model, coupling_map=coupling_map, basis_gates=basis_gates, shots=5000)
tomo_counts = tomo.tomography_data(job.result(), qst, efficient = True)
probs_psi, basis_matrix, _ = tomo.fitter_data(tomo_counts)

Full state tomography (maximum likelyhood approach)

In [14]:
rho_full_mle = tomo.state_mle_fit(probs_psi, basis_matrix)
eigenvalues, eigenvectors = np.linalg.eig(rho_full_mle)
print("Eigenvalues of rho: {}".format(eigenvalues))
print("Guess for psi: {}".format(eigenvectors[0]))
print("Fidelity: {}".format(inner_product(psi, eigenvectors[0])))

Eigenvalues of rho: [0.61503424+2.33335725e-18j 0.27906543+3.59350947e-18j
 0.06576326+1.04517168e-17j 0.04013707+1.13769921e-17j]
Guess for psi: [ 0.97271824+0.j         -0.22019051+0.03628985j  0.0041302 +0.05362442j
  0.02048342+0.02657468j]
Fidelity: 0.8322923439481025


Pure state tomography (maximum likelyhood approach)

In [15]:
psi_guess,_ = pure_state_mle_fit(probs_psi, basis_matrix)
print("Guess for psi: {}".format(psi_guess))
print("Fidelity: {}".format(inner_product(psi, psi_guess)))

Guess for psi: [(0.9249576204453344+0.03477278030303103j), (-0.29740943318417273+0.0333625183250024j), (0.18466812869883759+0.042615969519614806j), (-0.11911770651022562+0.05976084784122051j)]
Fidelity: 0.8206981208924022
